# Experiments on the German Credit datasets
Old version: https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)
Corrected version: https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29 / https://www.kaggle.com/c/south-german-credit-prediction/overview/data-overview

Some meanings of the discrete/ordinal feature values in the old version were wrong.
For example, for feature "checking status",

(inferred by the old dataset) a data object's value is                    '1' in the old dataset meaning negative DM.
(by new dataset) However, it should have been value          '4' in the old dataset meaning no checking account

(inferred by the old dataset) a data object's value is                    **'2' in the new dataset** *meaning negative DM*
(by new dataset) However, it should have been value          '1' in the old dataset meaning no checking account

Whether using the encoding scheme in the old or the new dataset, the feature value should be corrected according to the true meaning. The procedures in this experiment are:
1. Encode the dataset using the new dataset's meaning,
2. According to the *meanings of the old dataset*, **find encoded number in the new dataset and modify the feature values**.
3. Train first on the data points by *meanings of the old dataset* to get the base NN
4. gradually train on the data points by meanings of the new dataset to get the shifted NNs.

In [35]:
# Basics
import numpy as np
import pandas as pd
import csv

# sci-kit learn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split,cross_validate
from util_scripts.convert import extract_sklearn_params, custom_nn_model
from joblib import dump, load
import gurobipy

pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

import warnings
warnings.filterwarnings('ignore')

from util_scripts.preprocessor import Preprocessor, min_max_scale
from util_scripts.utilexp import *
from util_scripts.utilcredit import *
from interval import *
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.random.set_seed(1)
np.random.seed(1)

CYAN_COL = '\033[96m'
BLUE_COL = '\033[94m'
RED_COL = '\033[91m'
GREEN_COL = '\033[92m'
YELLOW_COL = '\033[93m'
RESET_COL = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

### Process the old version dataset

In [36]:
df_old, df_old_mm, df_old_enc, preprocessor_old = load_old("./datasets/credit/old/german.data")

In [37]:
df_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   checking-status  1000 non-null   int64
 1   duration         1000 non-null   int64
 2   credit-history   1000 non-null   int64
 3   purpose          1000 non-null   int64
 4   amount           1000 non-null   int64
 5   savings          1000 non-null   int64
 6   employment       1000 non-null   int64
 7   rate             1000 non-null   int64
 8   sex-status       1000 non-null   int64
 9   guarantors       1000 non-null   int64
 10  residence        1000 non-null   int64
 11  property         1000 non-null   int64
 12  age              1000 non-null   int64
 13  installment      1000 non-null   int64
 14  housing          1000 non-null   int64
 15  num-credits      1000 non-null   int64
 16  job              1000 non-null   int64
 17  liable           1000 non-null   int64
 18  phone    

In [38]:
df_old_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 73 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   checking-status_0  1000 non-null   float64
 1   checking-status_1  1000 non-null   float64
 2   checking-status_2  1000 non-null   float64
 3   checking-status_3  1000 non-null   float64
 4   duration           1000 non-null   float64
 5   credit-history_0   1000 non-null   float64
 6   credit-history_1   1000 non-null   float64
 7   credit-history_2   1000 non-null   float64
 8   credit-history_3   1000 non-null   float64
 9   credit-history_4   1000 non-null   float64
 10  purpose_0          1000 non-null   float64
 11  purpose_1          1000 non-null   float64
 12  purpose_2          1000 non-null   float64
 13  purpose_3          1000 non-null   float64
 14  purpose_4          1000 non-null   float64
 15  purpose_5          1000 non-null   float64
 16  purpose_6          1000 n

In [39]:
display(pd.DataFrame(data=df_old_mm.values[-1].reshape(1, -1), columns=columns))
display(preprocessor_old.encode_one(df_old_mm.values[-1]))

,checking-status,duration,credit-history,purpose,amount,savings,employment,rate,sex-status,guarantors,residence,property,age,installment,housing,num-credits,job,liable,phone,foreign,good-credit
0,2.0,0.602941,1.0,2.0,0.238032,2.0,0.0,2.0,1.0,0.0,3.0,1.0,0.142857,2.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0


,checking-status_0,checking-status_1,checking-status_2,checking-status_3,duration,credit-history_0,credit-history_1,credit-history_2,credit-history_3,credit-history_4,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,amount,savings_0,savings_1,savings_2,savings_3,savings_4,employment_0,employment_1,employment_2,employment_3,employment_4,rate_0,rate_1,rate_2,rate_3,sex-status_0,sex-status_1,sex-status_2,sex-status_3,guarantors_0,guarantors_1,guarantors_2,residence_0,residence_1,residence_2,residence_3,property_0,property_1,property_2,property_3,age,installment_0,installment_1,installment_2,housing_0,housing_1,housing_2,num-credits_0,num-credits_1,num-credits_2,num-credits_3,job_0,job_1,job_2,job_3,liable_0,liable_1,phone_0,phone_1,foreign_0,foreign_1,good-credit
0,1.0,1.0,1.0,0.0,0.602941,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.238032,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.142857,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [40]:
Xo, yo = df_old_enc.drop(columns=['good-credit']), pd.DataFrame(df_old_enc['good-credit'])
SPLIT = .2
Xo_train, Xo_test, yo_train, yo_test = train_test_split(Xo, yo, stratify=yo, test_size=SPLIT, shuffle=True,
                                                        random_state=0)
print(f'DATASET SIZE : train = {Xo_train.shape} {yo_train.shape} / test = {Xo_test.shape} {yo_test.shape}')


DATASET SIZE : train = (800, 72) (800, 1) / test = (200, 72) (200, 1)


### Process the new version dataset


In [41]:
df_new, df_new_mm, df_new_enc, preprocessor_new = load_new("./datasets/credit/new/train.csv")

In [42]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 1 to 800
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   checking-status  800 non-null    int64
 1   duration         800 non-null    int64
 2   credit-history   800 non-null    int64
 3   purpose          800 non-null    int64
 4   amount           800 non-null    int64
 5   savings          800 non-null    int64
 6   employment       800 non-null    int64
 7   rate             800 non-null    int64
 8   sex-status       800 non-null    int64
 9   guarantors       800 non-null    int64
 10  residence        800 non-null    int64
 11  property         800 non-null    int64
 12  age              800 non-null    int64
 13  installment      800 non-null    int64
 14  housing          800 non-null    int64
 15  num-credits      800 non-null    int64
 16  job              800 non-null    int64
 17  liable           800 non-null    int64
 18  phone     

In [43]:
df_new_enc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 799 entries, 1 to 799
Data columns (total 73 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   checking-status_0  799 non-null    float64
 1   checking-status_1  799 non-null    float64
 2   checking-status_2  799 non-null    float64
 3   checking-status_3  799 non-null    float64
 4   duration           799 non-null    float64
 5   credit-history_0   799 non-null    float64
 6   credit-history_1   799 non-null    float64
 7   credit-history_2   799 non-null    float64
 8   credit-history_3   799 non-null    float64
 9   credit-history_4   799 non-null    float64
 10  purpose_0          799 non-null    float64
 11  purpose_1          799 non-null    float64
 12  purpose_2          799 non-null    float64
 13  purpose_3          799 non-null    float64
 14  purpose_4          799 non-null    float64
 15  purpose_5          799 non-null    float64
 16  purpose_6          799 non-null

In [44]:
display(pd.DataFrame(data=df_new_mm.values[-1].reshape(1, -1), columns=columns))
display(preprocessor_new.encode_one(df_new_mm.values[-1]))


,checking-status,duration,credit-history,purpose,amount,savings,employment,rate,sex-status,guarantors,residence,property,age,installment,housing,num-credits,job,liable,phone,foreign,good-credit
0,0.0,0.382353,2.0,2.0,0.335644,4.0,4.0,3.0,2.0,0.0,3.0,1.0,0.214286,2.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0


,checking-status_0,checking-status_1,checking-status_2,checking-status_3,duration,credit-history_0,credit-history_1,credit-history_2,credit-history_3,credit-history_4,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,amount,savings_0,savings_1,savings_2,savings_3,savings_4,employment_0,employment_1,employment_2,employment_3,employment_4,rate_0,rate_1,rate_2,rate_3,sex-status_0,sex-status_1,sex-status_2,sex-status_3,guarantors_0,guarantors_1,guarantors_2,residence_0,residence_1,residence_2,residence_3,property_0,property_1,property_2,property_3,age,installment_0,installment_1,installment_2,housing_0,housing_1,housing_2,num-credits_0,num-credits_1,num-credits_2,num-credits_3,job_0,job_1,job_2,job_3,liable_0,liable_1,phone_0,phone_1,foreign_0,foreign_1,good-credit
0,1.0,0.0,0.0,0.0,0.382353,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.335644,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.214286,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0


In [45]:
Xn, yn = df_new_enc.drop(columns=['good-credit']), pd.DataFrame(df_new_enc['good-credit'])
SPLIT = .2
# don't need train-test split. only assessed on first dataset

In [46]:
display(Xn)

,checking-status_0,checking-status_1,checking-status_2,checking-status_3,duration,credit-history_0,credit-history_1,credit-history_2,credit-history_3,credit-history_4,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,amount,savings_0,savings_1,savings_2,savings_3,savings_4,employment_0,employment_1,employment_2,employment_3,employment_4,rate_0,rate_1,rate_2,rate_3,sex-status_0,sex-status_1,sex-status_2,sex-status_3,guarantors_0,guarantors_1,guarantors_2,residence_0,residence_1,residence_2,residence_3,property_0,property_1,property_2,property_3,age,installment_0,installment_1,installment_2,housing_0,housing_1,housing_2,num-credits_0,num-credits_1,num-credits_2,num-credits_3,job_0,job_1,job_2,job_3,liable_0,liable_1,phone_0,phone_1,foreign_0,foreign_1
1,1.0,0.0,0.0,0.0,0.205882,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043964,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.035714,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,1.0,1.0,0.0,0.0,0.073529,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.140255,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.303571,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.117647,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032519,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.071429,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.117647,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.103004,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.357143,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,1.0,0.0,0.0,0.0,0.088235,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.109552,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.517857,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1.0,0.0,0.0,0.0,0.205882,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.399527,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.571429,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
796,1.0,0.0,0.0,0.0,0.205882,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.204468,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.250000,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
797,1.0,1.0,1.0,1.0,0.117647,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327336,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.160714,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
798,1.0,1.0,0.0,0.0,0.250000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.683944,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.196429,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0


## Train NN on the old dataset


In [47]:
# Randomdised search + 5-fold cross validation (default)
nn = MLPClassifier(learning_rate='adaptive', random_state=0)

# parameters
max_iter_vals = [int(i) for i in np.linspace(1000, 10000, 10)]
hidden_layer_sizes_vals = [(i) for i in range(5, 16)]
batch_size_vals = [8, 16, 32, 64]
learning_rate_init_vals = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05]

#distributions = dict(max_iter=max_iter_vals, hidden_layer_sizes=hidden_layer_sizes_vals)
distributions = dict(hidden_layer_sizes=hidden_layer_sizes_vals,
                     batch_size=batch_size_vals,
                     learning_rate_init=learning_rate_init_vals,
                     max_iter=max_iter_vals, )

#nns = RandomizedSearchCV(nn, distributions, scoring='accuracy')
#search = nns.fit(Xo, yo)
#print(search.best_params_)

In [48]:
clf = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=5, learning_rate_init=0.02, batch_size=32,
                    max_iter=7000, random_state=0)

# 5-fold cross validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score, f1_score, precision_score

scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf, Xo, yo, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.2f %s with a std of %0.2f" % (scores[name].mean(), name, scores[name].std()))

clf = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=5, learning_rate_init=0.02, batch_size=32,
                    max_iter=7000, random_state=0)

clf.fit(Xo_train, yo_train)
resres = clf.predict(Xo_test.values)
print('\n', classification_report(yo_test, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))
resres = clf.predict(Xo_train.values)
print('\n', classification_report(yo_train, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))


0.75 test_accuracy with a std of 0.02
0.70 test_precision_macro with a std of 0.02
0.67 test_recall_macro with a std of 0.02
0.68 test_f1_macro with a std of 0.02

                  precision    recall  f1-score   support

 bad credit (0)      0.571     0.533     0.552        60
good credit (1)      0.806     0.829     0.817       140

       accuracy                          0.740       200
      macro avg      0.688     0.681     0.684       200
   weighted avg      0.735     0.740     0.737       200


                  precision    recall  f1-score   support

 bad credit (0)      0.662     0.629     0.645       240
good credit (1)      0.844     0.863     0.853       560

       accuracy                          0.792       800
      macro avg      0.753     0.746     0.749       800
   weighted avg      0.790     0.792     0.791       800



In [49]:
resres = clf.predict(Xo_test.values)
print('\n', classification_report(yo_test, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))
resres = clf.predict(Xo_train.values)
print('\n', classification_report(yo_train, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))



                  precision    recall  f1-score   support

 bad credit (0)      0.571     0.533     0.552        60
good credit (1)      0.806     0.829     0.817       140

       accuracy                          0.740       200
      macro avg      0.688     0.681     0.684       200
   weighted avg      0.735     0.740     0.737       200


                  precision    recall  f1-score   support

 bad credit (0)      0.662     0.629     0.645       240
good credit (1)      0.844     0.863     0.853       560

       accuracy                          0.792       800
      macro avg      0.753     0.746     0.749       800
   weighted avg      0.790     0.792     0.791       800



In [50]:
# save the trained classifier
dump(clf, 'credit.joblib')

['credit.joblib']

# Experiments: computing counterfactuals

#### Procedures

These procedures are covered by UtilExp class

1. Train M on D1
2. Get delta-min, build M+ and M-: incrementally train M 5 times, using different 10% of D2 each time, then get the maximum inf-distance between the incremented models and M. Construct M+ and M- using delta-min
3. Get M2: incrementally train M on D2
4. Select test instances: randomly select 50 D1 instances to explain, clf(x)=0, desired class=1
5. Report metrics using each baseline

#### Metrics
- Proximity: normalised L1: "Scaling Guarantees for Nearest CEs" page 7
- Sparsity: L0
- Validity-delta: percentage of test instances that 1) have counterfactuals valid on m1, 2) counterfactuals valid on M+ and M- under delta_min
- Validity-m2: percentage of test instances that 1) have counterfactual(s), 2) these counterfactual(s) are all valid on both m1 and m2
- LOF: average LOF score

In [51]:
clf = load("credit.joblib")

In [52]:
util_exp = UtilExp(clf, Xo, yo, Xn, yn, columns, ordinal_features, discrete_features, continuous_features, preprocessor_old.feature_var_map, gap=0.04, num_test_instances=2000)
print(util_exp.delta_max)
print(util_exp.delta_min)

1.5372448287620384
0.054449288758308775


In [53]:
# save model trained on the whole dataset
m2 = copy.deepcopy(clf)
m2.partial_fit(Xn, yn)
util_exp.Mmax = m2

In [54]:
# pre-verification on the points soundness 
valids = util_exp.verify_soundness()
print(len(valids))

percentage of sound model changes: 0.30633802816901406
87


In [55]:
valids = util_exp.verify_soundness(update_test_instances=True)

percentage of sound model changes: 0.30633802816901406
test instances updated to sound (x, Delta) pairs, length: 50


In [56]:
input_size, n_layers, output_size, output_act, h_act, optimizer, params = extract_sklearn_params(clf)
tf_model = custom_nn_model(input_size, n_layers, params, output_size, h_act, output_act, optimizer)
     
# Set model weights
for k, v in params.items():
	tf_model.layers[k].set_weights(v)
        
tf_model.save('./models/credit.h5', save_format='h5')   

input_size, n_layers, output_size, output_act, h_act, optimizer, params = extract_sklearn_params(m2)
tf_model = custom_nn_model(input_size, n_layers, params, output_size, h_act, output_act, optimizer)
     
# Set model weights
for k, v in params.items():
	tf_model.layers[k].set_weights(v)
	
tf_model.save('./models/credit_retrained.h5', save_format='h5')  

original_model = tf.keras.models.load_model('./models/credit.h5', compile=False)
old_weights = {}
for l in range(1,len(original_model.layers)):
	old_weights[l] = original_model.layers[l].get_weights()


model_retrained = tf.keras.models.load_model('./models/credit_retrained.h5', compile=False)
retrained_weights = {}
for l in range(1,len(model_retrained.layers)):
	retrained_weights[l] = model_retrained.layers[l].get_weights()


max_diff = -1
for l in range(1,len(old_weights)):
	old_layer_weights = old_weights[l][0]
	new_retrained_weights = retrained_weights[l][0]

	difference = abs(old_layer_weights - new_retrained_weights)
	
	for list_weights in difference:
		max_distance = max(list_weights)
		if max_distance > max_diff:
			max_diff = max_distance

print("\nThe maximum distance between weights is:", max_diff)


The maximum distance between weights is: 1.2818451


### CFX computation: in the following cells we both compute the CFX using MILP and the proposed probabilistic APΔS approach. 

In [57]:
# OURS-ROBUST: compute CFX based on the n sound points discovered above
ours_robust_ces_apas = util_exp.run_ours_robust(approx=True)
util_exp.evaluate_ces(ours_robust_ces_apas)
cfxs_robust_apas = ours_robust_ces_apas
print(len(cfxs_robust_apas))

50it [00:13,  3.70it/s]


total computation time in s: 13.529253005981445
found: 1.0
average normalised L1: 0.028256931400971
average normalised L0: 0.09299999999999993
average lof score: 1.0
counterfactual validity: 1.0
delta validity: 0.34
m2 validity: 0.94
50


In [30]:
# OURS-ROBUST: compute CFX based on MILP
ours_robust_ces = util_exp.run_ours_robust(approx=False)
util_exp.evaluate_ces(ours_robust_ces)
cfxs_robust = ours_robust_ces
print(len(cfxs_robust))

50it [00:07,  6.85it/s]


total computation time in s: 7.301151990890503
found: 1.0
average normalised L1: 0.03143011105681782
average normalised L0: 0.08399999999999991
average lof score: 1.0
counterfactual validity: 1.0
delta validity: 1.0
m2 validity: 1.0
50


In [31]:
# double checking the robustness of the CFXs found by APΔS approach after retraining
tot_robust_cfx = len(cfxs_robust_apas)
robust_cfx_after_retrain = 0

for cfx in cfxs_robust_apas:
    if model_retrained(np.array(cfx.reshape(1,-1))) >= 0.5:
        robust_cfx_after_retrain += 1

print(f"Percentage CFXs robust after the retraing: {(robust_cfx_after_retrain/tot_robust_cfx)*100}%") 

Percentage CFXs robust after the retraing: 94.0%


In [32]:
tot_robust_cfx = len(cfxs_robust)
robust_cfx_after_retrain = 0

for cfx in cfxs_robust:
    if model_retrained(np.array(cfx.reshape(1,-1))) >= 0.5:
        robust_cfx_after_retrain += 1

print(f"Percentage CFXs robust after the retraing: {(robust_cfx_after_retrain/tot_robust_cfx)*100}%") 

Percentage CFXs robust after the retraing: 100.0%


Here we define some utility functions for APΔS approach

In [33]:
def estimate_robustness(model, delta, cfx, concretizations, use_biases=True, robustness=True):

    """
    Utility method for the estimation of the CFX (not) Δ-robustness in the INN.

    Returns:
    --------
        rate: float
            estimation of the CFX (not) Δ-robustness computed with 'concretizations' models concretizations from the INN
    """
    np.random.seed(1)
    # Store initial weights
    old_weights = {}
    for l in range(1,len(model.layers)):
        old_weights[l] = model.layers[l].get_weights()

    for _ in range(concretizations):
        
        #perturbated_weights = {}
        input_features = np.array(cfx)

        for l in range(1,len(old_weights)+1):
            layer_weights = old_weights[l][0]
            if use_biases: layer_biases  = old_weights[l][1]
            
            weights_perturbation = np.random.uniform(-delta, delta, layer_weights.shape)
            if use_biases: biases_perturbation = np.random.uniform(-delta, delta, layer_biases.shape)
           
            
            layer_weights = [layer_weights+weights_perturbation]

            if use_biases: 
                layer_biases = [layer_biases+biases_perturbation]
                preactivated_res = np.dot(input_features, layer_weights) + layer_biases
            else:
                preactivated_res = np.dot(input_features, layer_weights)

            if l != len(old_weights):
                #relu
                activated_res = np.maximum(0.0, preactivated_res)
            else:
                #sigmoid
                activated_res = 1/(1 + np.exp(-preactivated_res))
            
            input_features = activated_res
            
        if input_features < 0.5:
            return 0  
    
    return 1

def compute_delta_max_MILP(cfx, delta_init, verbose=False):
  
    lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta_init)))
    if lower < 0.5: return 0 # CFX not robust
        
    delta = delta_init
    while lower >= 0.5: # over-approx lower bound is >= 0.5, i.e., x results robust
        delta = 2*delta
        lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta)))
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Lower is: {lower}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta_new)))
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {lower}')
        
        if lower >= 0.5:
            delta_max = delta_new
        else:
            delta = delta_new



def compute_delta_max(model, cfx, delta_init, concretizations, use_biases=True, verbose=False):
  
    rate = estimate_robustness(model, delta_init, cfx, concretizations, use_biases)
    if rate != 1: return 0 # CFX not robust
        
    delta = delta_init
    while rate == 1: # for all the concretizations x results robust
        delta = 2*delta
        rate = estimate_robustness(model, delta, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Rate is: {rate}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        rate = estimate_robustness(model, delta_new, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {rate}')
        
        if rate == 1:
            delta_max = delta_new
        else:
            delta = delta_new
        

In [34]:
model = tf.keras.models.load_model('./models/credit.h5', compile=False)
alpha = 0.999
R = 0.995
concretizations = int(np.emath.logn(R, (1-alpha)))
delta_init = 0.0001
delta_AAAI = 0.054449288758308775

with open("full_results_credit.csv", mode='w', newline='') as file:

    csv_writer = csv.writer(file)
    csv_writer.writerow(["CFX", "AAAI δ_max", "Wilks δ_max", "MILP δ_max", "Difference"])

    wilks = []
    milp = []
    print( f"{CYAN_COL}Condifence α =(1-R^n)={(1-R**concretizations)*100}%, R={R*100}%, Concretizations(n)={concretizations}{RESET_COL}")

    # start computing the new deltas with the approximation
    for cfx in cfxs_robust:

        delta_max_sampling = compute_delta_max(model, cfx.reshape(1,-1), delta_init, concretizations,verbose=False)
        delta_max_MILP = compute_delta_max_MILP(cfx, delta_init,verbose=False)
        difference = abs(delta_max_sampling-delta_max_MILP)
        wilks.append(delta_max_sampling)
        milp.append(delta_max_MILP)
        csv_writer.writerow([cfx,  delta_AAAI, delta_max_sampling, delta_max_MILP, difference])

        print('δ_max =', delta_max_sampling)
        print('δ improvement w.r.t original MILP\'s δ is',difference)
        print("______________________________________________________________________________________")



    csv_writer.writerow([" "])
    csv_writer.writerow(["Mean MILP", "Mean Wilks", ])
    csv_writer.writerow([ np.mean(np.array(milp)), np.mean(np.array(wilks))])


Condifence α =(1-R^n)=99.89995272421471%, R=99.5%, Concretizations(n)=1378
δ_max = 0.2485
δ improvement w.r.t original MILP's δ is 0.1618
______________________________________________________________________________________
δ_max = 0.20475
δ improvement w.r.t original MILP's δ is 0.1351
______________________________________________________________________________________
δ_max = 0.18020000000000003
δ improvement w.r.t original MILP's δ is 0.12090000000000002
______________________________________________________________________________________
δ_max = 0.14590000000000003
δ improvement w.r.t original MILP's δ is 0.08250000000000002
______________________________________________________________________________________
δ_max = 0.2322
δ improvement w.r.t original MILP's δ is 0.14889999999999998
______________________________________________________________________________________
δ_max = 0.16870000000000002
δ improvement w.r.t original MILP's δ is 0.10800000000000001
____________________